# Main Thesis Code 

This code contains all the functions and classes needed to generate results for determining error coefficients using least squares regression analysis of simulated data of an accelerometer going down the test track.

## Import libraries
Following cell imports all the libraries need to run the support functions and classes. 

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# from classes_x import *
import numpy as np
from scipy import integrate
import pandas as pd
from sigfig import round
import os
from sklearn.linear_model import LinearRegression

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'browser'

## Accelerometer Class
### Description:
The acceleromerter class defines an accelerometer object that contains an error model and a simulate function. The simulate function applies the associated error model attribute of the accelerombeter object and outputs what the output of the accelerometer would be given specific acceleration inputs. 
#### ___init___(self):
The accelerometer class is initiatlized with an error model that determines the kind of error the accelerometer demonstrates. For the purpose of this effort only the scale-factor non-linearity terms were added as they are a major focus of recent sled testing efforts. These values are estimated values for strategic grade resonating beam accelerometers. The values below can be found in the table below. 

| Coefficient	|Value	      |Units	       |Description                          |
| :---          | :---:       | :---:          | :---                                |
|$K_0$	        |5	          |$\mu g$	             |Bias                                 |
|$K_1$		    |.005         |$\mu g/g $	         |Scale Factor Error                   |
|$K_2$	        |60.144       |$\mu \frac{g}{g^2}$   |Scale factor 2nd order non-linearity |
|$K_3$	        |0.0152	      |$\mu \frac{g}{g^3}$   |Scale factor 3rd order non-linearity |
|$K_4$	        |0.0058	      |$\mu \frac{g}{g^4}$   |Scale factor 4th order non-linearity |
|$K_5$	        |0.0023       |$\mu \frac{g}{g^5}$   |Scale factor 5th order non-linearity |

In [ ]:
class Accelerometer:
    
    def __init__(self):  # Default accelerometer characteristics
     
        self.g = 9.791807                     # Definition of g
        
        self.AccelModelCoef = {'K_1': 5            * 10**-6,      # Scale Factor (g/g) NEEDS UPDATED
                               'K_0': .005         * 10**-6,      # Bias (g)
                               'K_2': 61.14        * 10**-6,      # is second-order coefficient (g/g^2)
                               'K_3': 0.02         * 10**-6,      # is third-order coefficient  (g/g^3)
                               'K_4': 0.006        * 10**-6,      # is fourth-order coefficient (g/g^4)
                               'K_5': 0.0023       * 10**-6       # is fifth-order coefficient  (g/g^5)
                               }
        
        ## Other acceleromter error coefficients that could be added in the future.
        # self.K_0_asym = 0                   # Bias Asymmetry 
        # self.K_1_asym = 0                   # Scale Factor Asymmetry
        # self.K_oq = 0                       # Odd Quadratic Coefficient
        # self.omeg_o = 0                    # is misalignmet of the IA with respect to the OA
        # self.omeg_p = 0                    # is misalignmen of the IA with respect to the PA
        # self.K_ip = 0                      # is crosscoupling coefficient 
        # self.K_io = 0                      # is crosscoupling coefficient
        # self.K_po = 0                      # is crosscoupling coefficient
        # self.K_pp = 1.32E-4 * 10**-6       # is cross-axis nonlinearity coefficients
        # self.K_ppp = 2.10E-7 * 10**-6
        # self.K_pppp = 2.3E-10 * 10**-6
        # self.K_oo = 0                      # is cros-axis nonlinearity coefficients
        # self.K_spin = 0                    # is spin correction coefficient, equal to 
        # self.K_ang_accel = 0               # is angular acceleration coefficient
        

#### class *Accelerometer* 
#### function *simulate(self, a_i, n_start_idx, n_stop_idx)*

##### Description: 
The simulate function simulates the output of a single acceleromter given accelation $(A_i)$ in g's along it's input axis. The accelerometer error $(A_{err})$ is given by the below equation

$$A_{err}= K_0+K_1 A_{i}+K_2A_{i}^{2}+K_3A_{i}^{3}+K_4A_{i}^{4}+K_5A_{i}^{5}$$

To get the actual output of the acceleromter the computed error is converted to ($m/s^2$) then added to the original input acceleration

$$A_{sim} = g*(A_{err}) + A_{i}$$

In [ ]:
    def simulate(self,a_i,n_start_idx, n_stop_idx):
        """
        Starting with one dimensional error model. Outputs acceleration given
        true input acceleration. In the future errors caused by inputs along the pendulus axis (a_p) 
        and output axis (a_o) 
        could be added.
        """
        
        # Convert acceleration into g's since the error coefficients are defined in terms of g's. 
        g_i = a_i / self.g
        
        accel_model = [self.AccelModelCoef['K_1'] * (g_i),
                       self.AccelModelCoef['K_0'] * np.ones(len(g_i)),  
                       self.AccelModelCoef['K_2'] * (g_i**2), 
                       self.AccelModelCoef['K_3'] * (g_i**3), 
                       self.AccelModelCoef['K_4'] * (g_i**4), 
                       self.AccelModelCoef['K_5'] * (g_i**5)]
        
        # Add accelerometer error from each coefficient together and multiply by g then add original acceleration.
        a_x_Sim = self.g * sum(accel_model[n_start_idx:n_stop_idx]) + a_i
        
        
        return a_x_Sim

## Generate Reference Trajectory
### Description:
These set of functions are used to generate a sled test trajectory. 

##### importEGIData(Headers,filepath)
Imports data from a .csv file into columns titled using the inputed Headers list. 

In [ ]:
def importEGIData(Headers,filepath):
        
    if filepath == '':
        print('No file selected')
    else: 
        D = pd.read_csv(filepath , names = Headers) # Pull only first row from Excel File

    return D

###### lpf(x, omege_c, T):
This function filters inputted data using a first order low pass filter. This is used to smooth out accelerometer data collected from a real sled test for use as data to create the reference trajectory.
 - x = Input array.
 - omega_c = Cutoff frequency.
 - T = Sample time. 

In [ ]:
def lpf(x, omega_c, T):
    """Implement a first-order low-pass filter.
    
    The input data is x, the filter's cutoff frequency is omega_c 
    [rad/s] and the sample time is T [s].  The output is y.
    """
    N = np.size(x)
    y = x
    alpha = (2-T*omega_c)/(2+T*omega_c)
    beta = T*omega_c/(2+T*omega_c)
    for k in range(1, N):
        y[k] = alpha*y[k-1] + beta*(x[k]+x[k-1])
        
    return y 

###### generateReferenceTrajectory()
This function generates a reference trajectory given some inputted accleration data. For this implementation it takes in acceleration and velocity data from an EGI on board a sled test that took place at the HHSTT. 

generateReferenceTrajectory Steps:

Step 1: Import Data.
Real acceleration and velocity data was collected from an Embedded GPS/INS device mounted on a guidance sled test. All three axes of data is imported but only the values from the downtrack (X) axis is used in this implementation. 

Step 2: Cleaning data
The data is then trimmed to focus on the part of the sled test where the actual launch occurs. The EGI sat for hours prior to launch during calibration of the unit under test but this data isn't necessary for this investigation. Occasionaly the time series from the raw data had repeated times for sucessive data points and so a new time series with an even sample rate was created and aplied to the data set to create a more even referenct trajectory. The new time series for the data was created by taking the total duration of the trajetory and dividing it by the total number of data points in the trajectory to get the new sample rate.

Step 3: Smoothing Data


In [ ]:
def generateReferenceTrajectory(plotcheck = False):
    
    # Define the file paths for acceleration and velocity data. 
    Accel_filepath = './EGI_data/EGI_accel.csv'
    Vel_filepath = './EGI_data/EGI_accel.csv'
    
    # Save data into Pandas data frames with defined headers.
    EGI_accel = importEGIData(['Time', 'Ax','Ay','Az'],Accel_filepath)
    EGI_vel = importEGIData(['Time', 'Vx','Vy', 'Vz'],Vel_filepath)

    # Combine the Acceleration and Velocity data frames into one.
    EGI_accel_vel = EGI_accel.join(EGI_vel[['Vx','Vy','Vz']])

    
    
    # Truth Gen Step 2 - Clean Data
    
    # Trim data to focus on actual sled run. These points were determined visually from the data used. 
    print('Developing Reference Trajectory')
    print("Trimming data to start/stop time determined visually...")
    startTime = 399600   # Index of data of the beginning of the reference trajectory
    stopTime = 399700    # Index of data of the end of the reference tracjectory

    # Trim the reference trajectory to the start and stop indicies defined above.  
    EGI_accel_vel_trim = EGI_accel_vel[(EGI_accel_vel['Time'] > startTime) & (EGI_accel_vel['Time'] < stopTime) ] # trim accelerometer output

    # The data used for creating the reference trajectory had repeated time values for multiple measurements of velocitty and acceleration.
    # To create a smooth reference trajectory the below code creates a new time series for the data by taking the total duration of the trajetory
    # and dividing it by the total number of data points in the trajectory so the new sample rate is even across the whole trajetory.
    
    # Determine new time series parameters
    Tdur = EGI_accel_vel_trim['Time'].max() - EGI_accel_vel_trim['Time'].min()
    Tlen = len(EGI_accel_vel_trim['Time'])

    # Generate new time series given duration of trajectory and number of data points.
    NewTimeSeries = np.linspace(0, Tdur, Tlen)
    
    # Save new time series to Data Frame.
    EGI_accel_vel_trim.loc[:,'New Time'] = NewTimeSeries
                         
        
    #%% Truth Gen Step 3 - Smooth Acceleration in X-axis
    
    # Pull data from data frame
    EGI_accel_presmoothed = EGI_accel_vel_trim[['Ax']]
    
    
    EGI_accel_smoothed_array = lpf(EGI_accel_vel_trim[['Ax']].to_numpy(),50,Tdur/Tlen)

    EGI_accel_vel_trim['Ax'] = EGI_accel_presmoothed

    # EGI_accel_vel_trim['Ax_smooth'] = pd.Series(EGI_accel_smoothed_array)

    #%% Truth Gen Step 4 - Create a DataFrame to house all truth data

    referenceTrajectory = pd.DataFrame()

    referenceTrajectory['Time'] = EGI_accel_vel_trim['New Time']
    referenceTrajectory['refAccel_x'] = EGI_accel_smoothed_array
    referenceTrajectory['refEGIVel_x'] = EGI_accel_vel_trim['Vx']

    # Create New Time Series
    referenceTrajectory['Time'] = np.linspace(0, Tdur, Tlen)

    # Change initial acceleration in X to zero until launch. Determined visually
    print("Setting initial acceleration to 0 until launch...")
    referenceTrajectory['refAccel_x'][:1145] = 0

    # Change final acceleration after stop to zero. Determined visually
    print("Setting final acceleration at 0...")
    referenceTrajectory['refAccel_x'][4968:] = 0
    
    
    #%% Truth Gen Step 5 -  Integrate truth acceleration to get velocity and distance
    referenceTrajectory['refVel_x'] = integrate.cumulative_trapezoid(y = referenceTrajectory['refAccel_x'],x = referenceTrajectory['Time'],initial = 0) 
    
    # Change final Velocity after stop to zero. Determined visually
    print("Setting final velocity at 0...")
    referenceTrajectory['refVel_x'][4968:] = 0
    
    referenceTrajectory['refDist_x'] = integrate.cumulative_trapezoid(y = referenceTrajectory['refVel_x'],x = referenceTrajectory['Time'],initial = 0) 


    # Integrate EGI velocity to compare to double integrated acceleration
    referenceTrajectory['refEGIDist_x'] = integrate.cumulative_trapezoid(y = referenceTrajectory['refEGIVel_x'],x = referenceTrajectory['Time'],initial = 0) 
    
    # Compute start motion time.
    startMotionTime = referenceTrajectory['Time'][referenceTrajectory['refAccel_x']>0.001].iloc[0]
    
    referenceTrajectory['Time'] = referenceTrajectory['Time']-startMotionTime
    
    
    #%% Save trajectory to Pickle File
    
    referenceTrajectory.to_pickle("./referenceTrajectory.pkl")
    
    #%% Plots Acceleration and Velocity
    if plotcheck == True:
        Figure1 = PlotlyPlot()
        
        Figure1.setTitle('EGI Acceleration, Velocity and Smoothed acceleration')
        Figure1.setYaxisTitle('Acceleration (m/s/s)')
        Figure1.setYaxis2Title('Velocity (m/s)')
        Figure1.setXaxisTitle('GPS Time (s)')
        Figure1.settwoAxisChoice([False, True])
        Figure1.plotTwoAxis(referenceTrajectory[['Ax','Vx']], df_x= EGI_accel_vel_trim[['New Time']])
        Figure1.addLine(referenceTrajectory[['refAccel_x']], df_x = referenceTrajectory[['Time']],secondary_y=False)
        Figure1.show()
    
    
    return 

#%% Generate Track RPV Function 

def generateTrackRPV(referenceTrajectory, sigmaRPV, tauRPV, biasRPV, Overwrite=True):
    
    print("\n Generating RPV")
    trackRPV = pd.DataFrame()
    
    # trackRPVzeroVel = "NoZeroVel"
    trackRPVzeroVel = 'NoZeroVel'
    
    
    if trackRPVzeroVel == "NoZeroVel":
        print("No zero velocity portions of test selected")
    
    Interupter_delta = 4.5 * 0.3048 # ft converted to meters
    TrackLength = 10000   # Meters
    
    trackRPV['Interupters_DwnTrk_dist'] = np.arange(0, TrackLength, Interupter_delta)
    
    trackRPV['Time'] = np.interp(trackRPV['Interupters_DwnTrk_dist'],referenceTrajectory['refDist_x'],referenceTrajectory['Time'])
    
    trackRPV = trackRPV[trackRPV['Interupters_DwnTrk_dist'] <= referenceTrajectory['refDist_x'].max()]
    
    trackRPV = trackRPV.drop_duplicates(subset=['Time'])
    
    trackRPV = trackRPV[:-1]
    
    if trackRPVzeroVel != "NoZeroVel":
        
        trackRPV_zeroVel_start = pd.DataFrame() 
        trackRPV_zeroVel_start['Time'] = referenceTrajectory['Time'][referenceTrajectory['Time']<trackRPV['Time'].min()]
        trackRPV_zeroVel_start['Interupters_DwnTrk_dist'] = 0
        trackRPV_zeroVel_start = trackRPV_zeroVel_start.tail(2)
    
        trackRPV_zeroVel_end = pd.DataFrame()
        
        trackRPV_zeroVel_end['Time'] = referenceTrajectory['Time'][referenceTrajectory['refVel_x']==0]
        trackRPV_zeroVel_end['Time'] = trackRPV_zeroVel_end['Time'][trackRPV_zeroVel_end['Time']>trackRPV['Time'].max()]
        trackRPV_zeroVel_end['Interupters_DwnTrk_dist'] = referenceTrajectory['refDist_x'].max()
        trackRPV_zeroVel_end = trackRPV_zeroVel_end.dropna()
        
        trackRPV_zeroVel_StartEnd = pd.DataFrame()
        trackRPV_zeroVel_StartMidEnd = pd.DataFrame()
        trackRPV_zeroVel_StartMid = pd.DataFrame()
    
        if trackRPVzeroVel == 'StartEnd':
            trackRPV_zeroVel_StartEnd = pd.concat((trackRPV_zeroVel_start,trackRPV_zeroVel_end), axis = 0)
            trackRPV_zeroVel_StartMidEnd = pd.concat((trackRPV, trackRPV_zeroVel_StartEnd), axis = 0)
            trackRPV_zeroVel_StartMidEnd = trackRPV_zeroVel_StartMidEnd.sort_values(by='Time').reset_index(drop=True)
            # trackRPV_zeroVel_StartMidEnd.to_pickle("./trackRPV_0Vel_StartEnd.pkl")
        elif trackRPVzeroVel == 'Start':
            trackRPV_zeroVel_StartMid = pd.concat((trackRPV, trackRPV_zeroVel_start), axis = 0)
            trackRPV_zeroVel_StartMid = trackRPV_zeroVel_StartMid.sort_values(by='Time').reset_index(drop=True)    
            # trackRPV_zeroVel_StartMid.to_pickle("./trackRPV_0Vel_Start.pkl")
            trackRPV = trackRPV_zeroVel_StartMid
    
    
    trackRPV = trackRPV.sort_values(by='Time').reset_index(drop=True)
    
    # trackRPV['Time'] = trackRPV['Time']-trackRPV['Time'][0]
    
    # Add error to Track RPV
    if sigmaRPV != 0:
        noise = np.random.normal(0,sigmaRPV,len(trackRPV)) # Add random noise to RPV
        trackRPV['Interupters_DwnTrk_dist'] = trackRPV['Interupters_DwnTrk_dist'] + noise
    
    if tauRPV != 0:
        trackRPV['Time'] = trackRPV['Time'] - tauRPV
        
    if biasRPV != 0:
        trackRPV['Interupters_DwnTrk_dist'] = trackRPV['Interupters_DwnTrk_dist'] + biasRPV


    #%% Save track RPV to pickle file
    if Overwrite == True:
        trackRPV.to_pickle(f"./RPVs/trackRPV_sig{sigmaRPV}_tau{tauRPV}_bias{biasRPV}.pkl")
    else:
       filepath = incrementFileName(f"./VarianceRPVs/trackRPV_sig{sigmaRPV}_tau{tauRPV}_bias{biasRPV}.pkl")
       trackRPV.to_pickle(filepath)
    return


def AccelSim(referenceTrajectory, N_model, changeDefaultCoeff, CoeffDict, g):
    
    #%% ACCEL SIM Step 1 - Simulate a Acceleromter with Bias using Accelerometer class
    """
    ACCEL SIM - Scripts used to generate simulated accelerometer output based on truth input
    
    Using smoothed acceleration truth data to simulate
    """
    
    AccelOne = Accelerometer()
    
    if changeDefaultCoeff == True:
            AccelOne.AccelModelCoef.update(CoeffDict)
    
    AccelOne.g = g
    
    # Create data frame to house data
    sensorSim = pd.DataFrame()
    sensorSim['Time'] = referenceTrajectory['Time']
    
    # Change to array for us in simulation.
    A_i_true = referenceTrajectory['refAccel_x'].to_numpy()  
    
    # Simulate
    A_x_sim = AccelOne.simulate(A_i_true, N_model[0], N_model[1])  
    
    #Store data in data frame. 
    sensorSim['SensorSim_Ax'] = A_x_sim
    
    sensorSim['SensorSim_Ax'][referenceTrajectory['refAccel_x'] == 0] = 0
    
    #%% Integrate Simulated accelerations to develop Velocity and Displacement.
    sensorSim['SensorSim_Vx'] = integrate.cumulative_trapezoid(y = sensorSim['SensorSim_Ax'],x = sensorSim['Time'],initial = 0) 
    sensorSim['SensorSim_Dx'] = integrate.cumulative_trapezoid(y = sensorSim['SensorSim_Vx'],x = sensorSim['Time'],initial = 0) 
    
    AccelObj = AccelOne
    
    return [sensorSim, AccelObj]

def RegressionAnalysis(referenceTrajectory, trackRPV, AccelObj, sensorSim, N_model, g,sigmaRPV, saveToPickel = False, WLSoption = True, LeastSquaresMethod = 'LongHand' ):
    
    #%% Error - Compare simulated acceleromter with track reference
    """
    Error - Scripts used to compare accelerometer simulation versus track truth
    """
    Dist_Error = pd.DataFrame()
    Dist_Error['Time'] = trackRPV['Time']
    
    # Interpolate Sensor Sim to Track
    
    trackRPV['SensorDwnTrkDist'] = np.interp(trackRPV['Time'],sensorSim['Time'],sensorSim['SensorSim_Dx'])
    
    Dist_Error['DistErr_x'] = trackRPV['Interupters_DwnTrk_dist'] - trackRPV['SensorDwnTrkDist']
    

    # Compute Velocity Error
    Ve_x = (np.diff(Dist_Error['DistErr_x'])/np.diff(Dist_Error['Time']))
    Ve_t = (Dist_Error['Time'].head(-1) + np.diff(Dist_Error['Time'])/2).to_numpy()
    
    Error = pd.DataFrame()
    
    Error['Time'] = Ve_t
    Error['SensorSim_Ax'] = np.interp(Ve_t,sensorSim['Time'],sensorSim['SensorSim_Ax']) 
    Error['SensorSim_Vx'] = np.interp(Ve_t,sensorSim['Time'],sensorSim['SensorSim_Vx'])
    Error['SensorSim_Dx'] = np.interp(Ve_t,sensorSim['Time'],sensorSim['SensorSim_Dx'])
    Error['DistErr_x'] = np.interp(Ve_t,Dist_Error['Time'],Dist_Error['DistErr_x']) 
    Error['VelErr_x'] = Ve_x
     
    #%% - Regression Analysis
    """
    Regression Analysis - Scripts used to compute error model
    """
    
    # # Compute coordinate functions
    referenceTrajectory['Ax^2 (g)'] = (referenceTrajectory[['refAccel_x']]/g)**2
    referenceTrajectory['Ax^3 (g)'] = (referenceTrajectory[['refAccel_x']]/g)**3
    referenceTrajectory['Ax^4 (g)'] = (referenceTrajectory[['refAccel_x']]/g)**4
    referenceTrajectory['Ax^5 (g)'] = (referenceTrajectory[['refAccel_x']]/g)**5
    
    referenceTrajectory['intAx^2 (g)'] = -integrate.cumulative_trapezoid(y = referenceTrajectory['Ax^2 (g)'],x = referenceTrajectory['Time'],initial = 0) 
    referenceTrajectory['intAx^3 (g)'] = -integrate.cumulative_trapezoid(y = referenceTrajectory['Ax^3 (g)'],x = referenceTrajectory['Time'],initial = 0) 
    referenceTrajectory['intAx^4 (g)'] = -integrate.cumulative_trapezoid(y = referenceTrajectory['Ax^4 (g)'],x = referenceTrajectory['Time'],initial = 0)
    referenceTrajectory['intAx^5 (g)'] = -integrate.cumulative_trapezoid(y = referenceTrajectory['Ax^5 (g)'],x = referenceTrajectory['Time'],initial = 0) 
    
    
    Vx = np.interp(Ve_t, referenceTrajectory['Time'],referenceTrajectory['refVel_x'])
    intAx_2 = np.interp(Ve_t,referenceTrajectory['Time'],referenceTrajectory['intAx^2 (g)']) 
    intAx_3 = np.interp(Ve_t,referenceTrajectory['Time'],referenceTrajectory['intAx^3 (g)']) 
    intAx_4 = np.interp(Ve_t,referenceTrajectory['Time'],referenceTrajectory['intAx^4 (g)']) 
    intAx_5 = np.interp(Ve_t,referenceTrajectory['Time'],referenceTrajectory['intAx^5 (g)'])
    
    coordinateFunctionDF = pd.DataFrame()
    coordinateFunctionDF['Time'] = Ve_t
     
    
    coeff_dict = {'Est_V_0': 0, 'Est_K_1': 0, 'Est_K_0': 0, 'Est_K_2': 0, 'Est_K_3': 0, 'Est_K_4': 0, 'Est_K_5': 0}
    
    # Create Complete A Matrix
    complete_A = np.array([np.ones(len(Ve_t))/g, -Vx/g, -Ve_t, intAx_2, intAx_3, intAx_4, intAx_5])*g
    complete_A = complete_A.T
    
    complete_A_DF = pd.DataFrame(np.fliplr(complete_A), columns=['IntAx_5', 'IntAx_4', 'IntAx_3', 'IntAx_2', 'Ve_t', 'Vx', 'Ones'])
    
    trimmed_A_filt = np.zeros(complete_A.shape[1], dtype = bool)
    trimmed_A_filt[0] = 1
    
    trimmed_A_filt[N_model[0]+1:N_model[1]+1] = 1

    trimmed_A = complete_A[:,trimmed_A_filt]
    
    '''
    COMPUTE COVARIANCE
    '''

    #%% Compute Covariance    
    
    #%% 
    # Linear Regression
    coeff_list = tuple(None for _ in range(trimmed_A.shape[1]))

    
    if sigmaRPV == 0 or WLSoption == False: 
        size = trimmed_A.shape[0]
        W = np.identity(size)
    
    else: 
        
        # Develop weighted matrix
        delta_t = np.diff(trackRPV['Time'])
        vel_sig = np.sqrt(2)*sigmaRPV/delta_t
        
        W = np.diag(vel_sig,0) - np.diag((.5*vel_sig[1:]),-1) - np.diag((.5*vel_sig[1:]),1)
    
        W = np.linalg.inv(W)
        
        # W = np.diag(1/vel_sig) 
        
    A = trimmed_A
    
    AW = np.transpose(trimmed_A).dot(W)
    Ve_xW = W.dot(Ve_x)
    

   
    if LeastSquaresMethod == 'Numpy':

        coeff_list = np.linalg.lstsq(np.transpose(AW), Ve_xW, rcond=None)[0] # This has just been used for debugging to check if "Long" least squares leads to same results.
    
    elif LeastSquaresMethod == 'SciKit':
        testSKlearn = LinearRegression()
        testSKlearn.fit(trimmed_A, Ve_x, sample_weight=(np.diag(W)))
        coeff_list = testSKlearn.coef_
        coeff_list[0] = testSKlearn.intercept_
    
    elif LeastSquaresMethod == 'LongHand':
        At = np.transpose(trimmed_A)
        coeff_list = np.linalg.inv(At.dot(W).dot(trimmed_A)).dot(At).dot(W).dot(Ve_x)
        
    else: 
        print("Did not select an applicable Least Squares Method")


    covariance_A = np.linalg.inv(np.dot(AW,trimmed_A))

    print_List = np.array(list(coeff_dict.keys()))
    
    n = 0
    for coef in print_List[trimmed_A_filt]:
        coeff_dict[coef] = coeff_list[n]
        n += 1
    
    #%% Save results to DataFrame
    
    coefficientDF = pd.DataFrame()
    
    coefficientDF = pd.concat((coefficientDF, pd.DataFrame.from_dict(AccelObj.AccelModelCoef, orient = 'index', columns= ['Accel Model'])))
    
    coefficientDF.loc['V_0'] = 0
    
    # Build Estimated Coefficient DF
    estimatedCoefficients = pd.DataFrame.from_dict(coeff_dict, orient = 'index', columns= ['Estimated Coefficients'])
    
    renameDict = {}
    for coeff in print_List:
        renameDict[coeff] = coeff[4:]
        
    estimatedCoefficients = estimatedCoefficients.rename(index = renameDict) 
    estimatedCoefficients.replace(0, np.nan, inplace=True)
    
    
    coefficientDF = pd.merge(coefficientDF,estimatedCoefficients,left_index=True, right_index=True)
    
    coefficientDF['Coefficient Estimate Error'] = coefficientDF['Accel Model'] - coefficientDF['Estimated Coefficients']
    
                
    #%% Compute Velocity Error Residuals
    
    V_error_model_terms = [coeff_dict['Est_V_0'], 
                           coeff_dict['Est_K_1']*Vx,  
                           coeff_dict['Est_K_0']*Ve_t, 
                           coeff_dict['Est_K_2']*intAx_2, 
                           coeff_dict['Est_K_3']*intAx_3,  
                           coeff_dict['Est_K_4']*intAx_4,  
                           coeff_dict['Est_K_5']*intAx_5]
    
    Error['V_error_model'] = sum(V_error_model_terms)*g 
    Error['Ve_x_Resid'] = Error['VelErr_x'] - Error['V_error_model'] 
    
  
    #%% Save off results:
    if saveToPickel == True:
        Error.to_pickle(f"./ErrorDF_{N_model[0]}-{N_model[1]}.pkl")
        coefficientDF.to_pickle(f"./coefficientDF_{N_model[0]}-{N_model[1]}.pkl")
        

    return [coefficientDF, Error, covariance_A, A, Ve_x, W, LeastSquaresMethod]

def figText(text):

    LaTeXText = '$\\text{' + text + ' }$'

    return LaTeXText

def round_array_to_sigfigs(array, sigfigs):
    rounded_array = np.zeros_like(array)  # Create an array of zeros with the same shape as the input array
    
    for i in range(array.shape[0]):
        for j in range(array.shape[1]):
            if array[i, j] == 0:
                rounded_array[i, j] = 0
            else:
                rounded_array[i, j] = round(array[i, j], sigfigs-1-int(np.floor(np.log10(np.abs(array[i, j])))))  # Calculate the number of decimals based on significant figures
    
    return rounded_array


def incrementFileName(base_path):
    
    # initialize the increment variable
    increment = 0
    
    # loop until we find a file name that doesn't exist

    while True:
        # create the file path with the increment
        file_path = f"{os.path.splitext(base_path)[0]}_{increment}{os.path.splitext(base_path)[1]}"
    
        # check if the file exists
        if os.path.isfile(file_path):
            # if it does, increment the counter and try again
            increment += 1
        else:
            # if it doesn't, break out of the loop
            break

    return file_path

## PlotlyPlot class
#### Description:

This plotly class is just a helper class that makes ploting Plotly plots easier. 

In [ ]:
def plotSimple(df, x = None, y = None):
    
    if x == None and y == None:
        fig = px.line(df)
        fig.show()
    elif y == None:
        fig = px.line(df, x = x)
        fig.show()
    else:
        fig = px.line(df, x = x, y = y)
        fig.show()    
    
    return

class PlotlyPlot:
        
    def __init__(self):
        
        self.title = ''
        self.x_axis = ''
        self.y_axis = ''
        self.y_axis_2 = ''
        self.twoAxisChoice = [False,True]
        self.template = 'simple_white'
        
    def setXaxisTitle(self,title):
        # if title[0] != '$':
        #     self.x_axis = self.figText(title)
        # else:        
        #     self.x_axis = title
        # return
    
        self.x_axis = title
    
    def setYaxisTitle(self,title):
        # if title[0] != '$':
        #     self.y_axis = self.figText(title)
        # else:        
        #     self.y_axis = title
        # return
    
        self.y_axis = title
    
    def setYaxis2Title(self,title):
        # if title[0] != '$':
        #     self.y_axis_2 = self.figText(title)
        # else:        
        #     self.y_axis_2 = title
        # return
    
        self.y_axis_2 = title
    
    
    def setTitle(self,title):
        # if title[0] != '$':
        #     self.title = self.figText(title)
        # else:        
        #     self.title = title
        # return
    
        self.title = title
    
    def settwoAxisChoice(self,twoAxisChoice):
        self.twoAxisChoice = twoAxisChoice
        return
    
    def plotNoDF(self, X = np.empty((0,)), Y = np.empty((0,)), Mode = 'lines', Name = None, Opacity = 1, Size = None):
        
        if X.size == 0:
            self.fig = go.Figure(go.Scatter(y = Y, name = Name, mode = Mode,  opacity = Opacity))
        else:
            self.fig = go.Figure(go.Scatter(x = X, y = Y, name = Name, mode = Mode,  opacity = Opacity))

        # Add Title 
        self.fig.update_layout(
            title_text = self.title)
        
        # Add Axis Labels
        self.fig.update_xaxes(title_text = self.x_axis)
        self.fig.update_yaxes(title_text = self.y_axis)
        
        return
    
    def addScatterNoDF(self,X = np.empty((0,)), Y = np.empty((0,)), Mode = 'lines', Name = None, Opacity = 1, Size = None, secondary_y = None):
        
        if secondary_y != None:           
            self.twoAxisChoice.append(secondary_y)
        
        if X.size == 0:
            self.fig.add_trace(go.Scatter(y = Y, name = Name, mode = Mode,  opacity = Opacity), secondary_y=secondary_y)
        else:
            self.fig.add_trace(go.Scatter(x = X, y = Y, name = Name, mode = Mode,  opacity = Opacity), secondary_y=secondary_y)


    def plotSimple(self,df, x = None, y = None):
        
        if x == None and y == None:
            self.fig = px.line(df)
            self.fig.show()
        elif y == None:
            self.fig = px.line(df, x = x)
            self.fig.show()
        else:
            self.fig = px.line(df, x = x, y = y)
            self.fig.show()    
            
        return

    def plotTwoAxis(self, df, df_x, Mode = 'lines', Name = None, Opacity = 1, Size = None):
        
        #df is a dataframe
        #LeftRight is a list of booleans that determine which y data gets plotted on second axis
        
        self.fig = make_subplots(specs=[[{"secondary_y": True}]])

        
        count = 0     
       
        for col in df:
            # Add Traces
            if Name != None: 
                self.fig.add_trace(
                    go.Scatter(x = df_x.iloc[:,0], y = df[col], name = Name, mode = Mode,  opacity = Opacity),
                    secondary_y = self.twoAxisChoice[count],)
            else:
                self.fig.add_trace(
                    go.Scatter(x = df_x.iloc[:,0], y = df[col], name = col, mode = Mode,  opacity = Opacity),
                    secondary_y = self.twoAxisChoice[count],)
            
            count += 1
        
        #Change size of markers
        if Size != None:
            self.update_marker_size(Size)
            
        # Add Title 
        self.fig.update_layout(
            title_text = self.title)
        
        # Add Axis Labels
        self.fig.update_xaxes(title_text = self.x_axis)
        self.fig.update_yaxes(title_text = self.y_axis, secondary_y = False)
        self.fig.update_yaxes(title_text = self.y_axis_2, secondary_y = True)

        return
    
    def addScatter(self,df, df_x, secondary_y = None, Name = None, Mode = 'markers', Opacity = 1, Size = None):
        
        if Name == None:
            Name = df.columns.values[0]
            
        if secondary_y != None:
            self.twoAxisChoice.append(secondary_y)
            self.fig.add_trace(go.Scatter(x = df_x.iloc[:,0],y = df.iloc[:,0], name = Name, mode = Mode, opacity = Opacity),secondary_y = secondary_y)
        else:
            self.fig.add_trace(go.Scatter(x = df_x.iloc[:,0],y = df.iloc[:,0], name = Name, mode = Mode, opacity=Opacity))
            
        #Change size of markers
        if Size != None:
            self.update_marker_size(Size)
    
    def addLine(self,df, df_x, secondary_y = None, Name = None, Opacity = 1):
        
        name = df.columns.values[0]
        
        if secondary_y != None:
            self.twoAxisChoice.append(secondary_y)
            self.fig.add_trace(go.Scatter(x = df_x.iloc[:,0],y = df.iloc[:,0], name = Name, opacity = Opacity),secondary_y = secondary_y)
        else:
            self.fig.add_trace(go.Scatter(x = df_x.iloc[:,0],y = df.iloc[:,0], name = Name, opacity = Opacity))
    
    def legendTopLeft(self):
        self.fig.update_layout(legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
            ))
        
    def legendTopRight(self):
        self.fig.update_layout(
            legend=dict(
                x=.9,
                y=1,
                xanchor='right',
                yanchor='top'
            ))
        
    def update_template(self, Template = 'simple_white'):
        self.fig.update_layout(template = Template)
        self.fig.update_layout(font=dict(family="Serif"))
        
    def update_legend(self):
        self.fig.update_layout(legend=dict(font=dict(family = 'Arial')))

    
    def write_image(self, figName, path):
        scale_factor = 1.2
        self.fig.write_image(f"{path}/{figName}.pdf", width = 600*scale_factor, height = 400*scale_factor)  
        self.fig.write_image(f"{path}/{figName}.png", width = 600*scale_factor, height = 400*scale_factor, scale=5)
        
    def addZoomSubPlot(self, zoom_x, zoom_y, Opacity=1):
        # zoom_x and zoom_y are the x and y coordinates of the new zoom window.
        # zoom_x = [x1, x2]
        # zoom_y = [y1, y2]
        
        #Initialize Subplot 
        traces = self.fig.data #Take all traces from first figure
        
        self.fig = make_subplots(rows=1, cols=2)
        
        colorsG10 = ['#3366CC', '#DC3912', '#FF9900', '#109618', '#990099', '#0099C6', '#DD4477', '#66AA00', '#B82E2E', '#316395']
        color_i = 0
        # Add all traces back into both figures
        
        for trace in traces:
            if trace.line != None:
                trace.update(opacity=Opacity, line=dict(color = colorsG10[color_i]))
                self.fig.add_trace(trace, row=1, col=1)
                self.fig.add_trace(trace, row=1, col=2)
                self.fig.data[-1].showlegend = False
                
            if color_i <= len(colorsG10):   
                color_i += 1
            else: 
                color_i = 0
                
        
        # Update zoom of subplot
        self.fig.update_xaxes(range=zoom_x, row=1, col=2)
        self.fig.update_yaxes(range=zoom_y, row=1, col=2)
        
        # Add box around zoom area
        self.addShadedBox(zoom_x, zoom_y, Row = 1, Col = 1)

   
    def addBox(self, box_x, box_y, Row=1, Col=1, scale_factor_x=1, scale_factor_y=1):
        
        box_X_scaled, box_Y_scaled = self.scale_rectangle(box_x, box_y, scale_factor_x,scale_factor_y)
        
        # Create the lines connecting the corners of the box to the corners of the second figure
        box_trace = go.Scatter(
            x=[box_X_scaled[0], box_X_scaled[0], box_X_scaled[1], box_X_scaled[1],box_X_scaled[0]],  # X coordinates for the lines
            y=[box_Y_scaled[0], box_Y_scaled[1], box_Y_scaled[1], box_Y_scaled[0],box_Y_scaled[0]], # Y coordinates for the lines
            mode='lines',
            line=dict(color='black', width=1),
            name = 'Zoomed Area' # Customize the line color, width, and style
        )
        self.fig.add_trace(box_trace, row = Row, col = Col) # Add box trace to original figure.
        
    def addShadedBox(self, box_x, box_y, Row=1, Col=1, scale_factor_x=1, scale_factor_y=1):
    
        box_X_scaled, box_Y_scaled = self.scale_rectangle(box_x, box_y, scale_factor_x,scale_factor_y)
        
        shape = go.layout.Shape(
            type="rect",
            xref="x",
            yref="y",
            x0=box_X_scaled[0],
            y0=box_Y_scaled[0],
            x1=box_X_scaled[1],
            y1=box_Y_scaled[1],
            fillcolor="lightblue",
            opacity=0.3,
            line=dict(color='black', width=1)
            
        )    
        
        self.fig.add_shape(shape,layer='below')
        
        self.addBox(box_x,box_y, Row=Row, Col=Col, scale_factor_x=scale_factor_x, scale_factor_y=scale_factor_y)
     
    def addLineShape(self, line_x, line_y, Row=1, Col=1):
        
        
        self.fig.add_shape(type="line",
                      x0=line_x[0], y0=line_y[0], x1=line_x[1], y1=line_y[1],
                      row=1, col=2,
                      line=dict(color="red", width=2))    
    
    def zoom(self, zoom_x, zoom_y, Row = 1, Col = 1):
        self.fig.update_xaxes(range=zoom_x, row=Row, col=Col)
        self.fig.update_yaxes(range=zoom_y, row=Row, col=Col)
    
    
    def show(self):
        self.fig.show()
        return
    
    def figText(self, text):

        LaTeXText = '$\\text{' + text + ' }$'

        return LaTeXText
    
    def update_marker_size(self, Size):
        
        #Get traces from Figure
        traces = self.fig.data
        import plotly.graph_objects as go

    def update_marker_size(self, marker_size):
  
        for data in self.fig.data:
            if 'marker' in data:
                data.marker.size = marker_size
                
        # self.fig.update_layout(legend= {'itemsizing': 'constant'})
       
    
    def scale_rectangle(self, x_coords, y_coords, scale_factor_x, scale_factor_y):
        x1 = x_coords[0]
        x2 = x_coords[1]
        y1 = y_coords[0]
        y2 = y_coords[1]
        
        # Calculating the center of the rectangle
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2
    
        # Calculating the width and height of the rectangle
        width = abs(x2 - x1)
        height = abs(y2 - y1)
    
        # Scaling up the rectangle
        new_width = width * scale_factor_x
        new_height = height * scale_factor_y
    
        # Calculating the new coordinates of the rectangle
        new_x1 = center_x - new_width / 2
        new_y1 = center_y - new_height / 2
        new_x2 = center_x + new_width / 2
        new_y2 = center_y + new_height / 2
        
        new_x_coords = [new_x1, new_x2]
        new_y_coords = [new_y1, new_y2]
    
        return new_x_coords, new_y_coords   